# 6.12.2. Función de tensión de Love en coordenadas cilíndricas

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez Marín, <daalvarez@unal.edu.co>  | Diciembre 11, 2023      | Código inicial |

## Función de tensión de Love en coordenadas cilíndricas

A continuación se definirá la función de tensión de Love para el caso de coordenadas cilíndricas a partir del vector de Galerkin.

In [1]:
from sympy import diff, Function, simplify, symbols, factor, Eq, init_printing, trigsimp, expand, latex, Matrix, zeros

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

Inicialmente definiremos algunas constantes y variables que necesitaremos para la definición de nuestro problema:

In [2]:
# constantes y variables
E, G,   = symbols("E, G")
r, t, z = symbols("r, theta, z")

# coeficiente de Poisson
nu = E/(2*G) - 1

# constante de Lamé
lamda = (nu*E)/((1 + nu)*(1 - 2*nu))

# vector de fuerzas másicas
b = zeros(3,1)

Definimos ahora los operadores básicos del cálculo vectorial en coordenadas cilíndricas:

In [3]:
# laplaciano de un campo escalar
lapl     = lambda f : (1/r)*diff(r*diff(f,r),r) + (1/(r**2))*diff(f,t,2) + diff(f,z,2)

# laplaciano vectorial en coordenadas cilíndricas
lapl_vec = lambda v : Matrix([ lapl(v[0]) - v[0]/(r**2) - (2/(r**2))*diff(v[1],t),
                               lapl(v[1]) - v[1]/(r**2) + (2/(r**2))*diff(v[0],t),
                               lapl(v[2])                                         ])

In [4]:
# gradiente de un campo escalar
grad = lambda f : Matrix([       diff(f,r),
                           (1/r)*diff(f,t),
                                 diff(f,z) ])

In [5]:
# divergencia de un campo vectorial
div = lambda v : (1/r)*diff(r*v[0],r) + (1/r)*diff(v[1],t) + diff(v[2],z)

Para definir la función de tensión de Love, haremos uso de un vector de Galerkin $\boldsymbol{p}$ de la forma:
\begin{equation*}
\boldsymbol{p}(r,\theta,z) = 
   \begin{pmatrix}
            0 \\
            0 \\
            \phi(r,\theta,z)
   \end{pmatrix}
\end{equation*}

In [6]:
phi = Function("phi")(r, t, z)
p   = Matrix([ 0, 0, phi ])

y con este definiremos el campo vectorial de desplazamientos $\boldsymbol{u}$ como:
\begin{equation*}
    \boldsymbol{u}(r,\theta,z) = 
    \begin{bmatrix} 
        u_r(r,\theta,z) \\ v_\theta(r,\theta,z) \\ w(r,\theta,z)
    \end{bmatrix}
 = \frac{2(1 - \nu)\nabla^2 \boldsymbol{p}(r,\theta,z) - \nabla(\nabla \cdot \boldsymbol{p}(r,\theta,z))}{2G}
\end{equation*}

In [7]:
# definición del campo vectorial de desplazamientos a partir del vector de Galerkin
vec_u = (2*(1 - nu)*lapl_vec(p) - grad(div(p)))/(2*G)
ur, vt, w = vec_u

Obteniendo así los desplazamientos:
\begin{align*}
    u_r(r,\theta, z)      &= - \frac{1}{2 G}   \frac{\partial^2 \phi(r,\theta,z)}{\partial r\partial z}\\
    v_\theta(r,\theta, z) &= - \frac{1}{2 G r} \frac{\partial^2 \phi(r,\theta,z)}{\partial \theta\partial z} \label{eq:urvtw_Galerkin}\\
    w(r,\theta, z)        &= \phantom{-}\frac{1}{2 G} \left(2(1 - \nu)\nabla^2 \phi(r,\theta, z) - \frac{\partial^2 \phi(r,\theta, z)}{\partial z^2}\right)
\end{align*}   

In [8]:
ur2 = -diff(phi, r, z)/(2*G)
vt2 = -diff(phi, t, z)/(2*G*r)
w2  =  (2*(1 - nu)*lapl(phi) - diff(phi, z, 2))/(2*G)

In [9]:
# Se verifica la equivalencia de las expresiones; imprime [0,0,0]
simplify(Matrix([ ur-ur2, vt-vt2, w-w2 ]))

Al reemplazar los desplazamientos en las ecuaciones:
\begin{align*}
 \varepsilon_r &= \frac{\partial u_r}{\partial r}                                            &  \gamma_{r\theta} &= \frac{1}{r}\frac{\partial u_r}{\partial \theta} + \frac{\partial v_\theta}{\partial r} - \frac{v_\theta}{r}  \\
 \varepsilon_\theta &= \frac{u_r}{r}  + \frac{1}{r}\frac{\partial v_\theta}{\partial \theta} & \gamma_{r z} &= \frac{\partial w}{\partial r} + \frac{\partial u_r}{\partial z} \\ 
 \varepsilon_z &= \frac{\partial w}{\partial z}                                              & \gamma_{\theta z} &= \frac{\partial v_\theta}{\partial z} + \frac{1}{r}\frac{\partial w}{\partial \theta}.
\end{align*}

In [10]:
er  = diff(ur, r)
et  = ur/r + (1/r)*diff(vt, t)
ez  = diff(w, z)
grt = (1/r)*diff(ur, t) + diff(vt, r) - vt/r
grz = diff(w, r) + diff(ur, z)
gtz = diff(vt, z) + (1/r)*diff(w, t)

se obtienen las deformaciones longitudinales:
\begin{alignat*}{2}
    \varepsilon_r      &= \frac{\partial u_r}{\partial r}                                                                             &&= - \frac{1}{2G}\frac{\partial^3 \phi}{\partial r^2\partial z} \\
    \varepsilon_\theta &= \frac{u_r}{r}  + \frac{1}{r}\frac{\partial v_\theta}{\partial \theta}                                       &&= - \frac{1}{2G}\frac{\partial}{\partial z}\left(\frac{1}{r}\frac{\partial\phi}{\partial r} + \frac{1}{r^2}\frac{\partial^2\phi}{\partial \theta^2}\right)\\
    \varepsilon_z      &= \frac{\partial w}{\partial z}                                                                               &&= \phantom{-}\frac{1}{2G}\frac{\partial}{\partial z}\left(2(1 - \nu)\nabla^2 \phi - \frac{\partial^2\phi}{\partial z^2}\right)
\end{alignat*}

In [11]:
er2 = -(1/(2*G))*diff(phi, r,r,z)
et2 = -(1/(2*G))*diff((1/r)*diff(phi, r) + (1/r**2)*diff(phi, t,2), z)
ez2 =  (1/(2*G))*diff(2*(1 - nu)*lapl(phi) - diff(phi, z,2), z)

In [12]:
# Se verifica la equivalencia de las expresiones; imprime [0,0,0]
simplify(Matrix([ er-er2, et-et2, ez-ez2 ]))

y las angulares:
\begin{alignat*}{2}
    \gamma_{r\theta}   &= \frac{1}{r}\frac{\partial u_r}{\partial \theta} + \frac{\partial v_\theta}{\partial r} - \frac{v_\theta}{r} &&= -\frac{1}{G}\frac{\partial^3}{\partial r \partial \theta \partial z}\left(\frac{\phi}{r}\right) \\
    \gamma_{rz}        &= \frac{\partial w}{\partial r} + \frac{\partial u_r}{\partial z}                                             &&= \phantom{-}\frac{1}{G} \frac{\partial}{\partial r}\left((1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2} \right)\\    
    \gamma_{\theta z}  &= \frac{\partial v_\theta}{\partial z} + \frac{1}{r}\frac{\partial w}{\partial \theta}                        &&= \phantom{-}\frac{1}{G r} \frac{\partial}{\partial \theta}\left((1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2}\right)
\end{alignat*}

In [13]:
grt2 = -(1/G)*diff(phi/r, r,t,z)
grz2 =  (1/G)*diff((1 - nu)*lapl(phi) - diff(phi, z,2), r)
gtz2 =  (1/(G*r))*diff((1 - nu)*lapl(phi) - diff(phi, z,2), t)

In [14]:
# Se verifica la equivalencia de las expresiones; imprime [0,0,0]
simplify(Matrix([ grt-grt2, grz-grz2, gtz-gtz2 ]))

Calculemos ahora la dilatación cúbica:

In [15]:
e = er + et + ez

para verificar que es:
\begin{equation*}
e = \varepsilon_r + \varepsilon_\theta + \varepsilon_z =\frac{1}{2 G} \frac{\partial}{\partial z}\left((1 - 2\nu) \nabla^2\phi\right)
\end{equation*}

In [16]:
e2 = (1/(2*G))*diff((1 - 2*nu)*lapl(phi),z)
expand(e - e2)

Aplicando las ecuaciones de Lamé:
\begin{align*}
   \sigma_r      &= \lambda e + 2 G \varepsilon_r        &   \tau_{r\theta} = G\gamma_{r\theta} \\
   \sigma_\theta &= \lambda e + 2 G \varepsilon_\theta   &   \tau_{rz} = G\gamma_{rz} \\
   \sigma_z      &= \lambda e + 2 G \varepsilon_z        &   \tau_{\theta z} = G\gamma_{\theta z}
\end{align*}

In [17]:
sr  = lamda*e + 2*G*er
st  = lamda*e + 2*G*et
sz  = lamda*e + 2*G*ez
trt = G*grt
trz = G*grz
ttz = G*gtz

obtenemos los esfuerzos son:
\begin{alignat*}{2}
    \sigma_r        &= \lambda e + 2 G \varepsilon_r      &&= \phantom{-}\frac{\partial}{\partial z}\left(\nu\nabla^2\phi - \frac{\partial^2 \phi}{\partial r^2}\right)\\
    \sigma_\theta   &= \lambda e + 2 G \varepsilon_\theta &&= \phantom{-}\frac{\partial}{\partial z}\left(\nu\nabla^2\phi - \frac{1}{r}\frac{\partial\phi}{\partial r} - \frac{1}{r^2}\frac{\partial^2\phi}{\partial \theta^2}\right)\\
    \sigma_z        &= \lambda e + 2 G \varepsilon_z      &&= \phantom{-}\frac{\partial}{\partial z}\left((2 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2}\right)\\
    \tau_{r\theta}  &= G\gamma_{r\theta}                  &&= -\frac{\partial^3}{\partial r \partial \theta \partial z}\left(\frac{\phi}{r}\right)\\
    \tau_{rz}       &= G\gamma_{rz}                       &&= \phantom{-}\frac{\partial}{\partial r}\left((1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2} \right)\\
    \tau_{\theta z} &= G\gamma_{\theta z}                 &&= \phantom{-}\frac{1}{r} \frac{\partial}{\partial \theta}\left((1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2} \right).
\end{alignat*}

In [18]:
sr2  =  diff(nu*lapl(phi) - diff(phi, r,2), z)
st2  =  diff(nu*lapl(phi) - (1/r)*diff(phi,r) - (1/r**2)*diff(phi, t,2), z)
sz2  =  diff((2 - nu)*lapl(phi) - diff(phi, z,2), z)
trt2 = -diff(phi/r, r,t,z)
trz2 =  diff((1 - nu)*lapl(phi) - diff(phi, z,2), r)
ttz2 =  (1/r)*diff((1 - nu)*lapl(phi) - diff(phi, z,2), t)

In [19]:
# Se verifica la equivalencia de las expresiones, imprime [0,0,0,0,0,0] */
simplify(Matrix([ sr-sr2, st-st2, sz-sz2, trt-trt2, trz-trz2, ttz-ttz2 ]))

## Función de tensión de Love en el caso axisimétrico

Observe que cuando $\phi$ es independiente de $\theta$, como en el caso de coordenadas axisimétricas, se tiene que los desplazamientos se simplifican a:

\begin{align*}
u_r(r,\theta, z)      &= - \frac{1}{2 G}   \frac{\partial^2 \phi(r,\theta,z)}{\partial r\partial z}\\
v_\theta(r,\theta, z) &= 0\\
w(r,\theta, z)        &= \frac{1}{2 G} \left(2(1 - \nu)\nabla^2 \phi(r,\theta, z) - \frac{\partial^2 \phi(r,\theta, z)}{\partial z^2}\right)
\end{align*}   

y los esfuerzos a:
\begin{align*}
\sigma_r        &= \frac{\partial}{\partial z}\left[\nu\nabla^2\phi - \frac{\partial^2 \phi}{\partial r^2}\right]\\
\sigma_\theta   &= \frac{\partial}{\partial z}\left[\nu\nabla^2\phi - \frac{1}{r}\frac{\partial\phi}{\partial r}\right]\\
\sigma_z        &= \frac{\partial}{\partial z}\left[(2 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2}\right]\\
\tau_{r \theta} &= 0 \\
\tau_{r z}      &= \frac{\partial}{\partial r}\left[(1 - \nu) \nabla^2\phi - \frac{\partial^2 \phi}{\partial z^2} \right]\\
\tau_{\theta z} &= 0
\end{align*}